<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://raw.githubusercontent.com/Arize-ai/phoenix-assets/9e6101d95936f4bd4d390efc9ce646dc6937fb2d/images/socal/github-large-banner-phoenix.jpg" width="1000"/>
        <br>
        <br>
        <a href="https://arize.com/docs/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://arize-ai.slack.com/join/shared_invite/zt-2w57bhem8-hq24MB6u7yE_ZF_ilOYSBw#/shared-invite/email">Community</a>
    </p>
</center>
<h1 align="center">Experiments: Text2SQL</h1>

Let's work through a Text2SQL use case where we are starting from scratch without a nice and clean dataset of questions, SQL queries, or expected responses.

In [1]:
!pip install "arize-phoenix>=10.0.0" openai 'httpx<0.28' duckdb datasets pyarrow "pydantic>=2.0.0" nest_asyncio openinference-instrumentation-openai --quiet

Let's first start a phoenix server. Note that this is not necessary if you have a phoenix server running already.

In [2]:
import phoenix as px

# px.launch_app().view()

/Users/mikeldking/work/phoenix/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Let's also setup tracing for OpenAI as we will be using their API to perform the synthesis.

In [1]:
from openinference.instrumentation.openai import OpenAIInstrumentor

from phoenix.otel import register

tracer_provider = register()
OpenAIInstrumentor().instrument(tracer_provider=tracer_provider)

/Users/mikeldking/work/phoenix/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: default
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: localhost:4317
|  Transport: gRPC
|  Transport Headers: {'user-agent': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  ⚠️ WARNING: It is strongly advised to use a BatchSpanProcessor in production environments.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



Let's make sure we can run async code in the notebook.

In [2]:
import nest_asyncio

nest_asyncio.apply()

Lastly, let's make sure we have our openai API key set up.

In [3]:
import os
from getpass import getpass

if not os.getenv("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass("🔑 Enter your OpenAI API key: ")

## Download Data

We are going to use the NBA dataset that information from 2014 - 2018. We will use DuckDB as our database.

In [4]:
import duckdb
from datasets import load_dataset

data = load_dataset("suzyanil/nba-data")["train"]

conn = duckdb.connect(database=":memory:", read_only=False)
conn.register("nba", data.to_pandas())

conn.query("SELECT * FROM nba LIMIT 5").to_df().to_dict(orient="records")[0]

{'Unnamed: 0': 1,
 'Team': 'ATL',
 'Game': 1,
 'Date': '10/29/14',
 'Home': 'Away',
 'Opponent': 'TOR',
 'WINorLOSS': 'L',
 'TeamPoints': 102,
 'OpponentPoints': 109,
 'FieldGoals': 40,
 'FieldGoalsAttempted': 80,
 'FieldGoals.': 0.5,
 'X3PointShots': 13,
 'X3PointShotsAttempted': 22,
 'X3PointShots.': 0.591,
 'FreeThrows': 9,
 'FreeThrowsAttempted': 17,
 'FreeThrows.': 0.529,
 'OffRebounds': 10,
 'TotalRebounds': 42,
 'Assists': 26,
 'Steals': 6,
 'Blocks': 8,
 'Turnovers': 17,
 'TotalFouls': 24,
 'Opp.FieldGoals': 37,
 'Opp.FieldGoalsAttempted': 90,
 'Opp.FieldGoals.': 0.411,
 'Opp.3PointShots': 8,
 'Opp.3PointShotsAttempted': 26,
 'Opp.3PointShots.': 0.308,
 'Opp.FreeThrows': 27,
 'Opp.FreeThrowsAttempted': 33,
 'Opp.FreeThrows.': 0.818,
 'Opp.OffRebounds': 16,
 'Opp.TotalRebounds': 48,
 'Opp.Assists': 26,
 'Opp.Steals': 13,
 'Opp.Blocks': 9,
 'Opp.Turnovers': 9,
 'Opp.TotalFouls': 22}

## Implement Text2SQL

Let's start by implementing a simple text2sql logic.

In [5]:
import os

import openai

client = openai.AsyncClient()

columns = conn.query("DESCRIBE nba").to_df().to_dict(orient="records")

# We will use GPT4o to start
TASK_MODEL = "gpt-4o"
CONFIG = {"model": TASK_MODEL}


system_prompt = (
    "You are a SQL expert, and you are given a single table named nba with the following columns:\n"
    f'{",".join(column["column_name"] + ": " + column["column_type"] for column in columns)}\n'
    "Write a SQL query corresponding to the user's request. Return just the query text, "
    "with no formatting (backticks, markdown, etc.)."
)


async def generate_query(input):
    response = await client.chat.completions.create(
        model=TASK_MODEL,
        temperature=0,
        messages=[
            {
                "role": "system",
                "content": system_prompt,
            },
            {
                "role": "user",
                "content": input,
            },
        ],
    )
    return response.choices[0].message.content

In [6]:
query = await generate_query("Who won the most games?")
print(query)

SELECT Team, COUNT(*) AS Wins
FROM nba
WHERE WINorLOSS = 'W'
GROUP BY Team
ORDER BY Wins DESC
LIMIT 1;


Awesome, looks like the LLM is producing SQL! let's try running the query and see if we get the expected results.

In [7]:
def execute_query(query):
    return conn.query(query).fetchdf().to_dict(orient="records")


execute_query(query)

[{'Team': 'GSW', 'Wins': 265}]

## Evaluation

Evaluation consists of three parts — data, task, and scores. We'll start with data.

In [8]:
questions = [
    "Which team won the most games?",
    "Which team won the most games in 2015?",
    "Who led the league in 3 point shots?",
    "Which team had the biggest difference in records across two consecutive years?",
    "What is the average number of free throws per year?",
]

Let's store the data above as a versioned dataset in phoenix.

In [10]:
import pandas as pd
import phoenix as px

# ds = px.Client().upload_dataset(
#     dataset_name="nba-questions",
#     dataframe=pd.DataFrame([{"question": question} for question in questions]),
#     input_keys=["question"],
#     output_keys=[],
# )

# If you have already uploaded the dataset, you can fetch it using the following line
ds = px.Client().get_dataset(name="nba-questions")

Next, we'll define the task. The task is to generate SQL queries from natural language questions.

In [11]:
async def text2sql(question):
    query = await generate_query(question)
    results = None
    error = None
    try:
        results = execute_query(query)
    except duckdb.Error as e:
        error = str(e)

    return {
        "query": query,
        "results": results,
        "error": error,
    }

Finally, we'll define the scores. We'll use the following simple scoring functions to see if the generated SQL queries are correct.

In [19]:
# Test if there are no sql execution errors


def invalid_sql(output):
    return 1.0 if output.get("error") is None else 0.0


# Test if the query has results
def valid(output):
    results = output.get("results")
    has_results = results is not None and len(results) > 0
    return 1.0 if has_results else 0.0

Now let's run the evaluation experiment.

In [23]:
import phoenix as px
from phoenix.experiments import run_experiment


# Define the task to run text2sql on the input question
def task(input):
    return text2sql(input["question"])

# Run the experiment    
experiment = run_experiment(
    ds, task=task, evaluators=[invalid_sql, valid], experiment_metadata=CONFIG
)

🧪 Experiment started.
📺 View dataset experiments: http://127.0.0.1:6006/datasets/RGF0YXNldDox/experiments
🔗 View this experiment: http://127.0.0.1:6006/datasets/RGF0YXNldDox/compare?experimentId=RXhwZXJpbWVudDozOA==



running experiment evaluations |██████████| 10/10 (100.0%) | ⏳ 00:03<00:00 |  2.76it/s
running tasks |██████████| 5/5 (100.0%) | ⏳ 00:06<00:00 |  1.34s/it






✅ Task runs completed.
🧠 Evaluation started.


running tasks |██████████| 5/5 (100.0%) | ⏳ 00:03<00:00 |  1.60it/s | ?it/s
running experiment evaluations |██████████| 10/10 (100.0%) | ⏳ 00:01<00:00 |  9.79it/s


🔗 View this experiment: http://127.0.0.1:6006/datasets/RGF0YXNldDox/compare?experimentId=RXhwZXJpbWVudDozOA==

Experiment Summary (06/06/25 05:50 PM -0600)
--------------------------------------------
     evaluator  n  n_scores  avg_score
0  invalid_sql  5         5        0.8
1        valid  5         5        0.6

Tasks Summary (06/06/25 05:50 PM -0600)
---------------------------------------
   n_examples  n_runs  n_errors
0           5       5         0


Ok! It looks like 3/5 of our queries are valid.


## Interpreting the results

Now that we ran the initial evaluation, it looks like two of the results are valid, two produce SQL errors, and one is incorrect.

- The incorrect query didn't seem to get the date format correct. That would probably be improved by showing a sample of the data to the model (e.g. few shot example).

- There are is a binder error, which may also have to do with not understanding the data format.

Let's try to improve the prompt with few-shot examples and see if we can get better results.

In [24]:
samples = conn.query("SELECT * FROM nba LIMIT 1").to_df().to_dict(orient="records")[0]
sample_rows = "\n".join(
    f"{column['column_name']} | {column['column_type']} | {samples[column['column_name']]}"
    for column in columns
)
system_prompt = (
    "You are a SQL expert, and you are given a single table named nba with the following columns:\n\n"
    "Column | Type | Example\n"
    "-------|------|--------\n"
    f"{sample_rows}\n"
    "\n"
    "Write a DuckDB SQL query corresponding to the user's request. "
    "Return just the query text, with no formatting (backticks, markdown, etc.)."
)


async def generate_query(input):
    response = await client.chat.completions.create(
        model=TASK_MODEL,
        temperature=0,
        messages=[
            {
                "role": "system",
                "content": system_prompt,
            },
            {
                "role": "user",
                "content": input,
            },
        ],
    )
    return response.choices[0].message.content


print(await generate_query("Which team won the most games in 2015?"))

SELECT Team, COUNT(*) AS Wins
FROM nba
WHERE WINorLOSS = 'W' AND Date LIKE '%/15'
GROUP BY Team
ORDER BY Wins DESC
LIMIT 1;


Looking much better! Finally, let's add a scoring function that compares the results, if they exist, with the expected results.




In [27]:
import phoenix as px
from phoenix.experiments import run_experiment


# Define the task to run text2sql on the input question
def task(input):
    return text2sql(input["question"])


experiment = run_experiment(
    ds, task=task, evaluators=[invalid_sql, valid], experiment_metadata=CONFIG
)

# Run the experiment
experiment.run()

# Get the results
results = experiment.results()

# Print the results

🧪 Experiment started.
📺 View dataset experiments: http://127.0.0.1:6006/datasets/RGF0YXNldDox/experiments
🔗 View this experiment: http://127.0.0.1:6006/datasets/RGF0YXNldDox/compare?experimentId=RXhwZXJpbWVudDo0MQ==


running tasks |██████████| 5/5 (100.0%) | ⏳ 00:02<00:00 |  1.82it/s

✅ Task runs completed.
🧠 Evaluation started.


running tasks |██████████| 5/5 (100.0%) | ⏳ 00:03<00:00 |  1.38it/s



🔗 View this experiment: http://127.0.0.1:6006/datasets/RGF0YXNldDox/compare?experimentId=RXhwZXJpbWVudDo0MQ==

Experiment Summary (06/06/25 05:51 PM -0600)
--------------------------------------------
     evaluator  n  n_scores  avg_score
0  invalid_sql  5         5        0.6
1        valid  5         5        0.6

Tasks Summary (06/06/25 05:51 PM -0600)
---------------------------------------
   n_examples  n_runs  n_errors
0           5       5         0


AttributeError: 'RanExperiment' object has no attribute 'run'

running experiment evaluations |██████████| 10/10 (100.0%) | ⏳ 00:02<00:00 |  4.79it/s


Amazing. It looks like we removed one of the errors, and got a result for the incorrect query. Let's try out using LLM as a judge to see how well it can assess the results.


In [52]:
from phoenix.evals.models import OpenAIModel
from phoenix.experiments import evaluate_experiment
from phoenix.experiments.evaluators.llm_evaluators import LLMCriteriaEvaluator

llm_evaluator = LLMCriteriaEvaluator(
    name="is_sql",
    criteria="is_sql",
    description="the output is a valid SQL query and that it executes without errors",
    model=OpenAIModel(),
)

evaluate_experiment(experiment, evaluators=[llm_evaluator])

🧠 Evaluation started.


running experiment evaluations |██████████| 5/5 (100.0%) | ⏳ 00:12<00:00 |  2.85s/it


🔗 View this experiment: http://127.0.0.1:6006/datasets/RGF0YXNldDox/compare?experimentId=RXhwZXJpbWVudDozMQ==

Experiment Summary (06/06/25 04:03 PM -0600)
--------------------------------------------
  evaluator  n  n_scores  avg_score
0    is_sql  5         5        0.6

Experiment Summary (06/06/25 04:03 PM -0600)
--------------------------------------------
     evaluator  n  n_scores  avg_score
0  has_results  5         5        0.6
1     no_error  5         5        0.6

Tasks Summary (06/06/25 04:03 PM -0600)
---------------------------------------
   n_examples  n_runs  n_errors
0           5       5         0


RanExperiment(id='RXhwZXJpbWVudDozMQ==', dataset_id='RGF0YXNldDox', dataset_version_id='RGF0YXNldFZlcnNpb246OQ==', repetitions=1)

Sure enough the LLM agrees with our scoring. Pretty neat trick! This can come in useful when it's difficult to define a scoring function.


We now have a simple text2sql pipeline that can be used to generate SQL queries from natural language questions. Since Phoenix has been tracing the entire pipeline, we can now use the Phoenix UI to convert the spans that generated successful queries into examples to use in **Golden Dataset** for regression testing!

<img src="https://storage.googleapis.com/arize-assets/phoenix/assets/images/golden_dataset.png">

## Generating more data
Now that we have a basic flow in place, let's generate some data. We're going to use the dataset itself to generate expected queries, and have a model describe the queries. This is a slightly more robust method than having it generate queries, because we'd expect a model to describe a query more accurately than generate one from scratch.




In [53]:
import json
from typing import List

from pydantic import BaseModel


class Question(BaseModel):
    sql: str
    question: str


class Questions(BaseModel):
    questions: List[Question]


sample_rows = "\n".join(
    f"{column['column_name']} | {column['column_type']} | {samples[column['column_name']]}"
    for column in columns
)
synthetic_data_prompt = f"""You are a SQL expert, and you are given a single table named nba with the following columns:

Column | Type | Example
-------|------|--------
{sample_rows}

Generate SQL queries that would be interesting to ask about this table. Return the SQL query as a string, as well as the
question that the query answers."""

response = await client.chat.completions.create(
    model="gpt-4o",
    temperature=0,
    messages=[
        {
            "role": "user",
            "content": synthetic_data_prompt,
        }
    ],
    tools=[
        {
            "type": "function",
            "function": {
                "name": "generate_questions",
                "description": "Generate SQL queries that would be interesting to ask about this table.",
                "parameters": Questions.model_json_schema(),
            },
        }
    ],
    tool_choice={"type": "function", "function": {"name": "generate_questions"}},
)

generated_questions = json.loads(response.choices[0].message.tool_calls[0].function.arguments)[
    "questions"
]
generated_questions[0]

running experiment evaluations |██████████| 5/5 (100.0%) | ⏳ 00:13<00:00 |  2.62s/it
running experiment evaluations |██████████| 10/10 (100.0%) | ⏳ 00:15<00:00 |  1.52s/it


{'sql': "SELECT Team, COUNT(*) AS Wins FROM nba WHERE WINorLOSS = 'W' GROUP BY Team ORDER BY Wins DESC;",
 'question': 'Which team has the most wins?'}

In [54]:
generated_dataset = []
for q in generated_questions:
    try:
        result = execute_query(q["sql"])
        generated_dataset.append(
            {
                "input": q["question"],
                "expected": {
                    "results": result,
                    "error": None,
                    "query": q["sql"],
                },
                "metadata": {
                    "category": "Generated",
                },
            }
        )
    except duckdb.Error as e:
        print(f"Query failed: {q['sql']}", e)
        print("Skipping...")

generated_dataset[0]

Query failed: SELECT Team, AVG(FieldGoals.) AS AvgFieldGoalPercentage FROM nba GROUP BY Team ORDER BY AvgFieldGoalPercentage DESC; Parser Error: syntax error at or near ")"
Skipping...


{'input': 'Which team has the most wins?',
 'expected': {'results': [{'Team': 'GSW', 'Wins': 265},
   {'Team': 'SAS', 'Wins': 230},
   {'Team': 'HOU', 'Wins': 217},
   {'Team': 'TOR', 'Wins': 215},
   {'Team': 'CLE', 'Wins': 211},
   {'Team': 'LAC', 'Wins': 202},
   {'Team': 'BOS', 'Wins': 196},
   {'Team': 'OKC', 'Wins': 195},
   {'Team': 'POR', 'Wins': 185},
   {'Team': 'WAS', 'Wins': 179},
   {'Team': 'UTA', 'Wins': 177},
   {'Team': 'ATL', 'Wins': 175},
   {'Team': 'IND', 'Wins': 173},
   {'Team': 'MIA', 'Wins': 170},
   {'Team': 'MEM', 'Wins': 162},
   {'Team': 'MIL', 'Wins': 160},
   {'Team': 'CHI', 'Wins': 160},
   {'Team': 'NOP', 'Wins': 157},
   {'Team': 'CHO', 'Wins': 153},
   {'Team': 'DET', 'Wins': 152},
   {'Team': 'DAL', 'Wins': 149},
   {'Team': 'DEN', 'Wins': 149},
   {'Team': 'MIN', 'Wins': 123},
   {'Team': 'SAC', 'Wins': 121},
   {'Team': 'ORL', 'Wins': 114},
   {'Team': 'NYK', 'Wins': 109},
   {'Team': 'PHI', 'Wins': 108},
   {'Team': 'BRK', 'Wins': 107},
   {'Team'

Awesome, let's crate a dataset with the new synthetic data.




In [55]:
synthetic_dataset = px.Client().upload_dataset(
    dataset_name="nba-golden-synthetic",
    inputs=[{"question": example["input"]} for example in generated_dataset],
    outputs=[example["expected"] for example in generated_dataset],
);

📤 Uploading dataset...


DatasetUploadError: Dataset with the same name already exists: name='nba-golden-synthetic'

In [21]:
run_experiment(
    synthetic_dataset, task=task, evaluators=[no_error, has_results], experiment_metadata=CONFIG
)

🧪 Experiment started.
📺 View dataset experiments: http://127.0.0.1:6006/datasets/RGF0YXNldDoxMg==/experiments
🔗 View this experiment: http://127.0.0.1:6006/datasets/RGF0YXNldDoxMg==/compare?experimentId=RXhwZXJpbWVudDoyMg==


running tasks |██████████| 8/8 (100.0%) | ⏳ 00:02<00:00 |  3.22it/s

✅ Task runs completed.
🧠 Evaluation started.



🔗 View this experiment: http://127.0.0.1:6006/datasets/RGF0YXNldDoxMg==/compare?experimentId=RXhwZXJpbWVudDoyMg==

Experiment Summary (06/06/25 03:43 PM -0600)
--------------------------------------------
     evaluator  n  n_scores  avg_score
0  has_results  8         8        0.5
1     no_error  8         8        0.5

Tasks Summary (06/06/25 03:43 PM -0600)
---------------------------------------
   n_examples  n_runs  n_errors
0           8       8         0


RanExperiment(id='RXhwZXJpbWVudDoyMg==', dataset_id='RGF0YXNldDoxMg==', dataset_version_id='RGF0YXNldFZlcnNpb246MTA=', repetitions=1)

Amazing! Now we have a rich dataset to work with and some failures to debug. From here, you could try to investigate whether some of the generated data needs improvement, or try tweaking the prompt to improve accuracy, or maybe even something more adventurous, like feed the errors back to the model and have it iterate on a better query. Most importantly, we have a good workflow in place to iterate on both the application and dataset.



# Trying a smaller model
Just for fun, let's wrap things up by trying out GPT-3.5-turbo. All we need to do is switch the model name, and run our Eval() function again.




In [22]:
TASK_MODEL = "gpt-3.5-turbo"

experiment = run_experiment(
    synthetic_dataset,
    task=task,
    evaluators=[no_error, has_results],
    experiment_metadata={"model": TASK_MODEL},
)

🧪 Experiment started.
📺 View dataset experiments: http://127.0.0.1:6006/datasets/RGF0YXNldDoxMg==/experiments
🔗 View this experiment: http://127.0.0.1:6006/datasets/RGF0YXNldDoxMg==/compare?experimentId=RXhwZXJpbWVudDoyMw==


running experiment evaluations |██████████| 16/16 (100.0%) | ⏳ 00:02<00:00 |  7.54it/s
running tasks |██████████| 8/8 (100.0%) | ⏳ 00:02<00:00 |  3.12it/s

✅ Task runs completed.
🧠 Evaluation started.


running tasks |██████████| 8/8 (100.0%) | ⏳ 00:04<00:00 |  1.62it/s



🔗 View this experiment: http://127.0.0.1:6006/datasets/RGF0YXNldDoxMg==/compare?experimentId=RXhwZXJpbWVudDoyMw==

Experiment Summary (06/06/25 03:43 PM -0600)
--------------------------------------------
     evaluator  n  n_scores  avg_score
0  has_results  8         8        1.0
1     no_error  8         8        1.0

Tasks Summary (06/06/25 03:43 PM -0600)
---------------------------------------
   n_examples  n_runs  n_errors
0           8       8         0


Interesting! It looks like the smaller model is able to do decently well but we might want to ensure it follows instructions as well as a larger model. We can actually grab all the LLM spans from our previous GPT40 runs and use them to generate a OpenAI fine-tuning JSONL file!

<img src="https://storage.googleapis.com/arize-assets/phoenix/assets/images/fine_tining_nba.png">
<img src="https://storage.googleapis.com/arize-assets/phoenix/assets/images/openai_ft.png">

## Conclusion

In this example, we walked through the process of building a dataset for a text2sql application. We started with a few handwritten examples, and iterated on the dataset by using an LLM to generate more examples. We used the eval framework to track our progress, and iterated on the model and dataset to improve the results. Finally, we tried out a less powerful model to see if we could save cost or improve latency.

Happy evaluations!